In [112]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [113]:
houses = pd.read_csv('data/cal_housing_clean.csv')

In [114]:
houses.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [115]:
houses = houses.apply(lambda x: (x - x.min()) / (x.max()-x.min() ))

In [116]:
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')
medianHouseValue = tf.feature_column.numeric_column('medianHouseValue')

feat_cols = [housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome]

In [117]:
x_data = houses.drop('medianHouseValue',axis=1)
label = houses['medianHouseValue']

In [118]:
X_train, X_test, y_train, y_test = train_test_split(x_data,label,test_size = 0.3)

In [119]:
# Training process

In [120]:
train_inputFunc = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=20,num_epochs=1000,shuffle=True)

In [121]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpj2msskfb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ddbdb3510>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [122]:
dnn_model.train(input_fn=train_inputFunc,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpj2msskfb/model.ckpt.
INFO:tensorflow:loss = 2.9170024, step = 1
INFO:tensorflow:global_step/sec: 777.093
INFO:tensorflow:loss = 0.4702669, step = 101 (0.130 sec)
INFO:tensorflow:global_step/sec: 764.121
INFO:tensorflow:loss = 0.54855764, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 761.54
INFO:tensorflow:loss = 0.2971897, step = 301 (0.131 sec)
INFO:tensorflow:global_step/sec: 930.597
INFO:tensorflow:loss = 0.7121575, step = 401 (0.108 sec)
INFO:tensorflow:global_step/sec: 920.987
INFO:tensorflow:loss = 0.7423308, step = 501 (0.107 sec)
INFO:tensorflow:global_step/sec: 955.562
INFO:tensorflow:loss = 0.44900382, step = 601 (0.105 sec)
INFO:tensorflow:global_step/sec: 962.036
INFO:tensorflow

INFO:tensorflow:global_step/sec: 870.253
INFO:tensorflow:loss = 0.29670662, step = 8001 (0.114 sec)
INFO:tensorflow:global_step/sec: 879.425
INFO:tensorflow:loss = 0.52295536, step = 8101 (0.114 sec)
INFO:tensorflow:global_step/sec: 1006.29
INFO:tensorflow:loss = 0.41721344, step = 8201 (0.099 sec)
INFO:tensorflow:global_step/sec: 768.176
INFO:tensorflow:loss = 0.24335009, step = 8301 (0.131 sec)
INFO:tensorflow:global_step/sec: 928.338
INFO:tensorflow:loss = 0.3388927, step = 8401 (0.107 sec)
INFO:tensorflow:global_step/sec: 888.217
INFO:tensorflow:loss = 0.39042023, step = 8501 (0.113 sec)
INFO:tensorflow:global_step/sec: 927.73
INFO:tensorflow:loss = 0.24257082, step = 8601 (0.108 sec)
INFO:tensorflow:global_step/sec: 865.221
INFO:tensorflow:loss = 0.58902407, step = 8701 (0.115 sec)
INFO:tensorflow:global_step/sec: 853.721
INFO:tensorflow:loss = 0.17100544, step = 8801 (0.119 sec)
INFO:tensorflow:global_step/sec: 892.128
INFO:tensorflow:loss = 0.2517549, step = 8901 (0.111 sec)
INF

In [123]:
# Prediction process

In [124]:
pred_inputFunc = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=20,num_epochs=1,shuffle=False)

In [125]:
pred_gen = dnn_model.predict(pred_inputFunc)

In [126]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpj2msskfb/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [127]:
final_preds =[]
for preds in predictions:
    final_preds.append(preds['predictions'])

In [128]:
mean_squared_error(y_test,final_preds)**0.5

0.1431414097058512

In [129]:
houses.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,0.541951,0.246776,0.0,0.333333,0.549020,0.705882,1.0
totalRooms,20640.0,0.066986,0.055486,0.0,0.036771,0.054046,0.080014,1.0
totalBedrooms,20640.0,0.083318,0.065371,0.0,0.045624,0.067349,0.100248,1.0
population,20640.0,0.039869,0.031740,0.0,0.021974,0.032596,0.048264,1.0
households,20640.0,0.081983,0.062873,0.0,0.045881,0.067094,0.099326,1.0
medianIncome,20640.0,0.232464,0.131020,0.0,0.142308,0.209301,0.292641,1.0
medianHouseValue,20640.0,0.395579,0.237928,0.0,0.215671,0.339588,0.514897,1.0
